## Purpose: Try different models-- Part5.
### Penalized_SVM.

In [16]:
# import dependencies.
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

#### STEP1: Read in dataset.  Remove data from 2016-2019.
- data from 2016-2018 will be used to bs test the model.
- data from 2019 will be used to predict the winners of the 2019 WS.

In [2]:
# read in the data.
team_data = pd.read_csv("../Resources/clean_data.csv")
del team_data["Unnamed: 0"]
team_data.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
0,St. Louis Cardinals,2019,1033,114,43,936,8313.0,3,2771,3847,...,0.319,456,4,895,33,3896,56,1.29,21,0
1,Arizona Diamondbacks,2019,1010,83,45,945,8538.0,2,2846,3901,...,0.315,472,7,925,24,4001,53,1.28,35,0
2,Kansas City Royals,2019,990,105,45,954,8421.0,6,2807,3842,...,0.346,543,5,816,24,4125,39,1.46,34,0
3,Houston Astros,2019,875,54,50,954,8589.0,6,2863,3788,...,0.284,432,7,1074,27,3929,67,1.14,31,0
4,Tampa Bay Rays,2019,975,92,53,963,8760.0,11,2920,3948,...,0.291,409,6,1037,26,3985,59,1.16,40,0


In [3]:
# remove data from 2016 through 2019.
team_data_new = team_data.loc[team_data["year"] < 2016]
team_data_new.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
120,San Francisco Giants,2015,1639,136,72,1458,13143.0,6,4381,6092,...,0.303,631,11,1309,43,6048,87,1.21,40,0
121,Washington Nationals,2015,1425,142,73,1458,13137.0,17,4379,5877,...,0.300,612,12,1476,46,6036,95,1.19,47,0
122,Houston Astros,2015,1599,135,77,1458,13212.0,18,4404,6080,...,0.314,701,8,1396,44,6180,84,1.29,98,0
123,Detroit Tigers,2015,1537,148,75,1449,12852.0,5,4284,5896,...,0.320,721,8,1232,47,6048,86,1.32,44,0
124,Boston Red Sox,2015,1427,139,75,1458,12957.0,37,4319,5821,...,0.314,694,5,1362,43,6073,93,1.27,52,0


In [4]:
target = team_data_new["winners"]
features = team_data_new.drop({"team", "year", "winners"}, axis=1)
feature_columns = list(features.columns)
print (target.shape)
print (features.shape)
print (feature_columns)

(2344,)
(2344, 46)
['A', 'DP', 'E', 'GS2', 'INN', 'PB', 'PO', 'TC', '2B', '3B', 'AB', 'AO', 'BB', 'H', 'HBP', 'HR', 'NP_x', 'OBP', 'OPS_x', 'R', 'RBI', 'SAC', 'SB', 'SLG', 'BB1', 'BK', 'CG', 'ER', 'ERA', 'G1', 'GF', 'GS', 'H1', 'HB', 'HR1', 'IP', 'L', 'OBP1', 'R1', 'SHO', 'SO1', 'SV', 'TBF', 'W', 'WHIP', 'WP']


#### STEP2: Split and scale the data.

In [6]:
# split data.
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

# scale data.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


#### STEP3: Try the SVC model.

In [7]:
# generate the model.
model = SVC(kernel="rbf",
           class_weight="balanced",
           probability=True)

# fit the model.
model.fit(X_train_scaled, y_train)

# predict.
prediction = model.predict(X_test_scaled)

print ((classification_report(y_test, prediction, target_names=["0", "1"])))

              precision    recall  f1-score   support

           0       0.96      0.79      0.87       553
           1       0.13      0.52      0.21        33

   micro avg       0.78      0.78      0.78       586
   macro avg       0.55      0.65      0.54       586
weighted avg       0.92      0.78      0.83       586



#### STEP4: Predict the winner 2016-2018.

In [9]:
def predict_the_winner(model, year, team_data, X_train):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -model = the saved model.
    -team_data = complete dataframe with all data.
    -year = the year want to look at.
    
    OUTPUT:
    -printed prediction.
    
    DESCRIPTION:
    -data from year of interest is isolated.
    -the data are scaled.
    -the prediction is made.
    -print out the resulting probability and the name of the team.
    '''
    
    # grab the data.
    team_data = team_data.loc[team_data["year"] == year].reset_index()

    # set features (no team, year, winners).
    # set target (winners).
    features = team_data[feature_columns]
    
    # scale.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    features = scaler.fit_transform(features)
    
    # fit the model.
    probabilities = model.predict_proba(features)

    # convert predictions to datafram.e
    WS_predictions = pd.DataFrame(probabilities[:,1])

    # Sort the DataFrame (descending)
    WS_predictions = WS_predictions.sort_values(0, ascending=False)

    WS_predictions['Probability'] = WS_predictions[0]

    # Print 50 highest probability HoF inductees from still eligible players
    for i, row in WS_predictions.head(50).iterrows():
       prob = ' '.join(('WS Probability =', str(row['Probability'])))
       print('')
       print(prob)
       print(team_data.iloc[i,1:27]["team"])

In [10]:
# predict for 2018.
predict_the_winner(model, 2018, team_data, X_train_scaled)


WS Probability = 0.15780822061260966
Arizona Diamondbacks

WS Probability = 0.10722959919030854
Minnesota Twins

WS Probability = 0.07655108394291248
Washington Nationals

WS Probability = 0.069485398364
Houston Astros

WS Probability = 0.0673887130576405
New York Yankees

WS Probability = 0.05950457665100013
Tampa Bay Rays

WS Probability = 0.057451586557511605
St. Louis Cardinals

WS Probability = 0.056833084593734956
Texas Rangers

WS Probability = 0.05463754982650527
Cincinnati Reds

WS Probability = 0.05314003732479712
Atlanta Braves

WS Probability = 0.05259206522814144
Oakland Athletics

WS Probability = 0.05237552490902348
Cleveland Indians

WS Probability = 0.04767185635180766
New York Mets

WS Probability = 0.04472360968549403
Boston Red Sox

WS Probability = 0.04403109395997146
Chicago Cubs

WS Probability = 0.03873493773652864
Milwaukee Brewers

WS Probability = 0.033564460585811036
Los Angeles Angels

WS Probability = 0.029218635885838572
Kansas City Royals

WS Probabilit

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [11]:
# predict for 2017.
predict_the_winner(model, 2017, team_data, X_train_scaled)


WS Probability = 0.1388687852068434
Atlanta Braves

WS Probability = 0.1291738875463083
Washington Nationals

WS Probability = 0.11332720124129604
Los Angeles Angels

WS Probability = 0.10250513347321008
Cleveland Indians

WS Probability = 0.08654035440596093
Boston Red Sox

WS Probability = 0.07315009268876517
Tampa Bay Rays

WS Probability = 0.07247618835386947
Oakland Athletics

WS Probability = 0.06452990209382294
New York Yankees

WS Probability = 0.06015152549348108
Houston Astros

WS Probability = 0.045290349371858524
Seattle Mariners

WS Probability = 0.04225378892771399
Milwaukee Brewers

WS Probability = 0.038650012497827096
Los Angeles Dodgers

WS Probability = 0.034679864020187155
Pittsburgh Pirates

WS Probability = 0.031750929034553475
Baltimore Orioles

WS Probability = 0.03166446618267395
Colorado Rockies

WS Probability = 0.02738868501951206
Arizona Diamondbacks

WS Probability = 0.02726680524308179
Kansas City Royals

WS Probability = 0.026077782092324153
Chicago Whi

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Ok.  This didn't work. Let's try this penalized model with a grid search.

In [19]:
def grid_search_svc(X_train, X_test, y_train, y_test):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -X_test = scaled X test data.
    -y_train = y train data.
    -y_test = y test data.
    
    OUTPUT:
    -classification report (has F1 score, precision and recall).
    -grid = saved model for prediction. 
    
    DESCRIPTION:
    -the scaled and split data is put through a grid search with svc.
    -the model is trained.
    -a prediction is made.
    -print out the classification report and give the model.
    '''
    
    # set up svc model.
    model = SVC(kernel="rbf", 
                class_weight="balanced",
                probability=True)

    # create gridsearch estimator.
    param_grid = {"C": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                 "gamma": [0.0001, 0.001, 0.01, 0.1]}
    grid = GridSearchCV(model, param_grid, verbose=3)

    # fit the model.
    grid.fit(X_train, y_train)

    # predict.
    prediction = grid.predict(X_test)
    
    # print out the basic information about the grid search.
    print (grid.best_params_)
    print (grid.best_score_)
    print (grid.best_estimator_)
    
    grid = grid.best_estimator_
    predictions = grid.predict(X_test)
    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    
    return grid

In [20]:

model_grid = grid_search_svc(X_train, X_test, y_train, y_test)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV] C=0.0001, gamma=0.0001 ..........................................
[CV]  C=0.0001, gamma=0.0001, score=0.044293015332197615, total=   0.5s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  C=0.0001, gamma=0.0001, score=0.04436860068259386, total=   0.5s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.9572649572649573, total=   0.5s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV]  C=0.0001, gamma=0.001, score=0.044293015332197615, total=   0.5s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] . C=0.0001, gamma=0.001, score=0.04436860068259386, total=   0.5s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.9572649572649573, total=   0.5s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] . C=0.0001, gamma=0.01, score=0.044293015332197615, total=   0.4s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] .. C=0.0001, gamma=0.01, score=0.04436860068259386, total=   0.4s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.9572649572649573, total=   0.4s
[CV] C=0.0001, gamma=0.1 .............................................
[CV] .

[CV] ..... C=10, gamma=0.0001, score=0.9557069846678024, total=   0.5s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.9556313993174061, total=   0.5s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.9572649572649573, total=   0.6s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.9557069846678024, total=   0.6s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.9556313993174061, total=   0.6s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.9572649572649573, total=   0.6s
[CV] C=10, gamma=0.01 ................................................
[CV] ....... C=10, gamma=0.01, score=0.9557069846678024, total=   0.4s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:   45.3s finished
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'C': 0.1, 'gamma': 0.001}
0.9562002275312855
SVC(C=0.1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       553
           1       0.00      0.00      0.00        33

   micro avg       0.94      0.94      0.94       586
   macro avg       0.47      0.50      0.49       586
weighted avg       0.89      0.94      0.92       586



/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Nope.  This is terrible.  Lots of no.